<a href="https://colab.research.google.com/github/Lexxos/AiVoice/blob/master/Copy_of_Trumpifyve_Run_Saved_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<a href="https://colab.research.google.com/github/JDBumgardner/Trumpifyve/blob/master/Trumpifyve_Run_Saved_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2019 The T5 Authors. All Rights Reserved.
# Copyright 2020 Jacob Bumgardner, Jack Robison, and Google inc. 
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

#### Description 

https://github.com/JDBumgardner/Trumpifyve/

In this notebook we demonstrate how to run inference on our pretrained models to trumpify arbitrary text. First we decide on a model temperature, then we load the saved and exported model and then we apply the style shift over input text. 


## Load the saved model

In [1]:
%tensorflow_version 2.x
!pip install tensorflow-text


     |████████████████████████████████| 4.9 MB 5.1 MB/s 


In [ ]:
MODEL_TEMPERATURE = "1.5" #@param["0", "1", "1.5", "2"]
saved_model_path = {
    "0": 'gs://style_shift_trump/models/3B/export/1587870976',
    "1": 'gs://style_shift_trump/models/3B/export/1587870500',
    "1.5":'gs://style_shift_trump/models/3B/export/1587870045',
    "2":'gs://style_shift_trump/models/3B/export/1587869196',
    }[MODEL_TEMPERATURE]


In [6]:
!apt upgrade -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Calculating upgrade... Done
The following packages were automatically installed and are no longer required:
  linux-headers-4.15.0-161 linux-headers-4.15.0-161-generic
Use 'apt autoremove' to remove them.
The following NEW packages will be installed:
  linux-headers-4.15.0-162 linux-headers-4.15.0-162-generic
The following packages have been kept back:
  libcudnn8 libcudnn8-dev libnccl-dev libnccl2
The following packages will be upgraded:
  base-files binutils binutils-common binutils-x86-64-linux-gnu gnupg2 gzip
  icu-devtools libaudit-common libaudit1 libbinutils libc-bin libcublas-dev
  libcublas10 libcudnn7 libcudnn7-dev libgnutls30 libhogweed4 libicu-dev
  libicu60 libiculx60 libldap-2.4-2 libldap-common liblz4-1 libnettle6
  libopenexr-dev libopenexr22 libp11-kit0 libpam-modules libpam-modules-bin
  libpam-runtime libpam0g libpq-dev libpq5 librados2 librbd1 libsasl2-2
  libsasl2-module

In [7]:
%tensorflow_version 2.x 
!pip install -q t5
import tensorflow as tf
import tensorflow_text  # Required to run exported model.

def load_predict_fn(model_path):
  if tf.executing_eagerly():
    print("Loading SavedModel in eager mode.")
    imported = tf.saved_model.load(model_path, ["serve"])
    return lambda x: imported.signatures['serving_default'](tf.constant(x))['outputs'].numpy()
  else:
    print("Loading SavedModel in tf 1.x graph mode.")
    tf.compat.v1.reset_default_graph()
    sess = tf.compat.v1.Session()
    meta_graph_def = tf.compat.v1.saved_model.load(sess, ["serve"], model_path)
    signature_def = meta_graph_def.signature_def["serving_default"]
    return lambda x: sess.run(
        fetches=signature_def.outputs["outputs"].name, 
        feed_dict={signature_def.inputs["input"].name: x}
    )

predict_fn = load_predict_fn(saved_model_path)

TensorFlow is already loaded. Please restart the runtime to change versions.
     |████████████████████████████████| 153 kB 5.0 MB/s 
     |████████████████████████████████| 366 kB 29.7 MB/s 
     |████████████████████████████████| 4.0 MB 51.1 MB/s 
     |████████████████████████████████| 90 kB 8.9 MB/s 
     |████████████████████████████████| 1.2 MB 40.0 MB/s 
     |████████████████████████████████| 286 kB 63.4 MB/s 
     |████████████████████████████████| 3.1 MB 38.2 MB/s 
     |████████████████████████████████| 3.3 MB 31.9 MB/s 
     |████████████████████████████████| 596 kB 55.3 MB/s 
     |████████████████████████████████| 895 kB 42.5 MB/s 
     |████████████████████████████████| 59 kB 7.0 MB/s 
     |████████████████████████████████| 489.6 MB 24 kB/s 
     |████████████████████████████████| 1.3 MB 45.8 MB/s 
     |████████████████████████████████| 463 kB 49.7 MB/s 
     |████████████████████████████████| 5.8 MB 46.8 MB/s 


ImportError: ignored

## Apply

We can now call the style_shift method.

In [ ]:
def style_shift(text_sample):
  return predict_fn(["trumpifyve:" + text_sample])[0].decode('utf-8')

list_of_text_samples = [
                        "Encourage your staff or community members to protect their personal health.",
                        "Post the signs and symptoms of COVID-19: fever, cough, shortness of breath.",
                        "Clean surfaces that are frequently touched – things such as shared desks, countertops, kitchen areas, electronics, and doorknobs.",
                        "Limit events and meetings that require close contact.",
                        "Stay up to date on developments in your community.",
                        "Create an emergency plan for possible outbreak.",
                        "Assess if community members are at higher risk and plan accordingly."
                 ]


for text_sample in list_of_text_samples:
    print(style_shift(text_sample))